In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd

import numpy as np
from matplotlib import pyplot as plt

from art.estimators.classification import SklearnClassifier
from art.attacks.evasion import ZooAttack
from art.utils import load_mnist
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score
import platform

import warnings
warnings.filterwarnings('ignore')

C:\Users\msi\anaconda3\envs\Shayekh_20230126\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cd "../"

C:\Users\msi\Desktop\Research lab\Covid_AI_Research_Lab\Machine_Learning


In [3]:
cd "../"

C:\Users\msi\Desktop\Research lab\Covid_AI_Research_Lab


In [4]:
#Reading the pre_processed data
os_type = platform.system()
if os_type.startswith("Darwin"):
    data = pd.read_excel(r"dataset/processed_data/main/merged_scored_dataset1.xlsx")
else: 
    data = pd.read_excel(r"dataset\processed_data\main\merged_scored_dataset1.xlsx")
#Droping the target col
feature_value_temp = data.drop("Laboratory confirmed, since the beginning of the pandemic Hospitalized", axis=1)
#Droping the Distric col as it is string and will be assigned value by clustering
features = feature_value_temp.drop("District",axis=1)
features = features.drop("Unnamed: 0",axis=1)
features = features.drop("Laboratory confirmed, since the beginning of the pandemic TOTAL",axis=1)
features = features.drop("Laboratory confirmed, since the beginning of the pandemic RecoveredA",axis=1)
features = features.drop("Laboratory confirmed, since the beginning of the pandemic Deceased",axis=1)


data['flag'] = 3  # Initialize the 'flag' column with 0
data.loc[data['Laboratory confirmed, since the beginning of the pandemic Hospitalized'] <= 449, 'flag'] = 2
data.loc[data['Laboratory confirmed, since the beginning of the pandemic Hospitalized'] <= 290, 'flag'] = 1

target_col = data["flag"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(features, target_col, test_size=0.3, random_state=29)

In [6]:
model = RandomForestClassifier(n_estimators=50, random_state= 22)
model.fit(X=X_train, y=y_train)

# Create ART classifier for scikit-learn RandomForestClassifier
art_classifier = SklearnClassifier(model=model)

# Create ART Zeroth Order Optimization attack
zoo = ZooAttack(classifier=art_classifier, confidence=0.0, targeted=False, learning_rate=1e-1, max_iter=20,
                binary_search_steps=10, initial_const=1e-3, abort_early=True, use_resize=False, 
                use_importance=False, nb_parallel=1, batch_size=1, variable_h=0.2)

 # Generate adversarial samples with ART Zeroth Order Optimization attack
x_train_adv = zoo.generate(X_train.values)

ZOO:   0%|          | 6/11690 [00:06<3:20:36,  1.03s/it]

KeyboardInterrupt



In [12]:
x_train_adv = np.load('adversarial_data.npy')

In [8]:
x_test_adv = zoo.generate(X_test.values)

ZOO: 100%|██████████| 5010/5010 [49:22<00:00,  1.69it/s]   


In [10]:
np.save('x_test_adv.npy', x_test_adv )

In [14]:
score = model.score(X_train, y_train)
print("Normal Training Score: %.4f" % score)

score = model.score(x_train_adv, y_train)
print("Adversarial Training Score: %.4f" % score)

score = model.score(X_test, y_test)
print("Normal Test Score: %.4f" % score)

score = model.score(x_test_adv, y_test)
print("Adversarial Test Score: %.4f" % score)

Normal Training Score: 0.9985
Adversarial Training Score: 0.9739
Normal Test Score: 0.7715
Adversarial Test Score: 0.7527


In [8]:
# # Create a mapping between column names and numbers
# def convert_to_numbers(df):
#     column_mapping = {col: i for i, col in enumerate(df.columns)}
#     df.rename(columns=column_mapping, inplace=True)

# convert_to_numbers(X_train)
# convert_to_numbers(X_test)

In [21]:
# X_train.shape

Root Mean Squared Error: 23.37818172449867
Mean Squared Error: 546.5393807436836
R2 Score: 0.9957503364591642


In [11]:
from art.estimators.classification import SklearnClassifier

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

#model = RandomForestClassifier(n_estimators=50, random_state= 22)
#model.fit(X_train,y_train)
# Make predictions
y_pred = model.predict(X_test)

# Calculate accuracy, recall, precision, and F1 score
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred,average='weighted')
precision = precision_score(y_test, y_pred,average='weighted')
f1 = f1_score(y_test, y_pred,average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Precision: {precision:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.77
Recall: 0.77
Precision: 0.76
F1 Score: 0.77


In [9]:
# #AttributeInferenceBlackBox Attacking
# import numpy as np
# from art.attacks.inference.attribute_inference import AttributeInferenceBlackBox

# attack_train_ratio = 0.5
# attack_train_size = int(len(X_train) * attack_train_ratio)
# attack_x_train = X_train[:attack_train_size].values
# attack_y_train = y_train[:attack_train_size]
# attack_x_test = X_train[attack_train_size:].values
# attack_y_test = y_train[attack_train_size:]

# attack_feature = 2  # sex

# # get original model's predictions
# attack_x_test_predictions = np.array([np.argmax(arr) for arr in art_regressor.predict(attack_x_test)]).reshape(-1,1)
# # only attacked feature
# attack_x_test_feature = attack_x_test[:, attack_feature].copy().reshape(-1, 1)
# # training data without attacked feature
# attack_x_test = np.delete(attack_x_test, attack_feature, 1)

# bb_attack = AttributeInferenceBlackBox(art_regressor, attack_feature=attack_feature, is_continuous = True)

# # train attack model
# bb_attack.fit(attack_x_train)


# # get inferred values
# values = [6084,  35802.5  ]
# inferred_train_bb = bb_attack.infer(attack_x_test, pred=attack_x_test_predictions)
# # check accuracy
# train_acc = np.sum(inferred_train_bb == np.around(attack_x_test_feature, decimals=8).reshape(1,-1)) / len(inferred_train_bb)
# print(train_acc)